In [1]:
import dask.dataframe as dd

In [2]:
# Função para processar em pandas
def import_prices(df):
    # Ordenar o DataFrame pelos campos necessários
    df = df.sort_values(by=['item', 'store_code', 'sell_price', 'yearweek'])

    # Preencher os valores NaN com o valor anterior
    df['yearweek'] = df['yearweek'].ffill()

    # Converter a coluna yearweek para inteiro
    df['yearweek'] = df['yearweek'].astype(int)

    # Função para garantir a unicidade de yearweek, incrementando se necessário
    def increment_yearweek(df):
        last_yearweek = None
        for i, row in df.iterrows():
            if last_yearweek is not None and row['yearweek'] <= last_yearweek:
                last_yearweek += 1
                df.at[i, 'yearweek'] = int(last_yearweek)
            else:
                last_yearweek = row['yearweek']
        return df

    # Aplicar a função para incrementar yearweek dentro de cada grupo de item, store_code, e sell_price
    df = df.groupby(['item', 'store_code', 'sell_price'], group_keys=False).apply(increment_yearweek).reset_index(drop=True)

    return df

# Carregar o arquivo item_prices.csv com Dask
item_prices = dd.read_csv('../data/item_prices.csv', dtype={'yearweek': 'float64'})

# Aplicar a função de pandas a cada partição do Dask DataFrame
df_filled_dask = item_prices.map_partitions(import_prices)

# Salvar como múltiplos arquivos Parquet
df_filled_dask.to_parquet('../data/prices/', write_index=False)

C:\Users\rpiaz\AppData\Local\Temp\ipykernel_13444\1476150606.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['item', 'store_code', 'sell_price'], group_keys=False).apply(increment_yearweek).reset_index(drop=True)
C:\Users\rpiaz\AppData\Local\Temp\ipykernel_13444\1476150606.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['item', 'store_code', 'sell_price'], group_k

In [5]:
df_filled_dask.head(100)

C:\Users\rpiaz\AppData\Local\Temp\ipykernel_13444\1476150606.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['item', 'store_code', 'sell_price'], group_keys=False).apply(increment_yearweek).reset_index(drop=True)


,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.9858
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201331,10.9858
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201332,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201333,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201334,10.9858
...,...,...,...,...,...
95,ACCESORIES_1_001,ACCESORIES,NYC_1,201519,10.9858
96,ACCESORIES_1_001,ACCESORIES,NYC_1,201520,10.9858
97,ACCESORIES_1_001,ACCESORIES,NYC_1,201521,10.9858
98,ACCESORIES_1_001,ACCESORIES,NYC_1,201522,10.9858


In [4]:
df_filled_dask.isna().sum().compute()

item          0
category      0
store_code    0
yearweek      0
sell_price    0
dtype: int64

In [6]:
row_201328 = df_filled_dask.loc[df_filled_dask['yearweek'] == 201328].compute()
print(row_201328)

                      item     category store_code  yearweek  sell_price
155       ACCESORIES_1_001   ACCESORIES      NYC_1    201328     12.7414
578       ACCESORIES_1_002   ACCESORIES      NYC_1    201328      5.2801
860       ACCESORIES_1_002   ACCESORIES      NYC_2    201328      5.2801
1145      ACCESORIES_1_002   ACCESORIES      NYC_3    201328      5.2801
1757      ACCESORIES_1_004   ACCESORIES      NYC_1    201328      6.1712
...                    ...          ...        ...       ...         ...
1393229  SUPERMARKET_3_824  SUPERMARKET      PHI_2    201328      3.2160
1393567  SUPERMARKET_3_824  SUPERMARKET      PHI_3    201328      3.2160
1393678  SUPERMARKET_3_825  SUPERMARKET      PHI_2    201328      4.7760
1393963  SUPERMARKET_3_825  SUPERMARKET      PHI_3    201328      4.7760
1394246  SUPERMARKET_3_826  SUPERMARKET      PHI_2    201328      1.5360

[24639 rows x 5 columns]
